# Trabalho prático AC

## Trabalho realizado por:

Margarida Vila-chã, 20210504

Miguel Duarte Silva, 20210504

Sebastião Lessa, 202103238


## Algoritmo escolhido e breve explicação:

O algoritmo escolhido pelo nosso grupo foi o Random Forrest.

Este algoritmo é um algoritmo de aprendizagem computacional supervisionado. Ele usa uma combinação de árvores de decisão aleatórias. O processo começa com a criação de um grande número de árvores (floresta), em que cada uma das árvores criadas é treinada com uma amostra diferente. Isto permite a criação de árvores suficientemente diferentes, o que leva a que elas juntas consigam lidar com uma ampla variedade de situações, reduzindo o risco de overffitting.

Baseamos-nos nos seguintes data-sets:

- [credit-g](https://www.openml.org/search?type=run&id=591&collections.id=99&run_task.task_id=31&sort=date)
- [breast-w](https://www.openml.org/search?type=run&id=7413&collections.id=99&run_task.task_id=15)
- [qsar-biodeg](https://www.openml.org/search?type=run&id=519587&collections.id=99&run_task.task_id=9957)

## Alteração escolhida

A alteração que o nosso grupo decidiu fazer no codigo fonte obtido foi a de considerar a utilização de outros tipos de critérios de divisão. O critério pré definido no nosso código fonte é o de entropia, por isso vamos testar com o gini e com o classification error.

## Import das bibliotecas necessárias:

In [60]:
from scipy.io import arff
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from RandomForest import RandomForest
from random import random
from sklearn import datasets
from sklearn.model_selection import train_test_split
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score



## Testes de benchmark com todos os data-sets

### DATASET 1 - Cancro da mama

In [61]:
dataset = pd.DataFrame(arff.loadarff('datasets/breast-w.arff')[0])

In [62]:
dataset.head()

,Clump_Thickness,Cell_Size_Uniformity,Cell_Shape_Uniformity,Marginal_Adhesion,Single_Epi_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,5.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,b'benign'
1,5.0,4.0,4.0,5.0,7.0,10.0,3.0,2.0,1.0,b'benign'
2,3.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0,b'benign'
3,6.0,8.0,8.0,1.0,3.0,4.0,3.0,7.0,1.0,b'benign'
4,4.0,1.0,1.0,3.0,2.0,1.0,3.0,1.0,1.0,b'benign'


In [63]:
for column in ['Class']:
    le = LabelEncoder()
    dataset[column] = le.fit_transform(dataset[column])

In [64]:
dataset.head()

,Clump_Thickness,Cell_Size_Uniformity,Cell_Shape_Uniformity,Marginal_Adhesion,Single_Epi_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,5.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,0
1,5.0,4.0,4.0,5.0,7.0,10.0,3.0,2.0,1.0,0
2,3.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0,0
3,6.0,8.0,8.0,1.0,3.0,4.0,3.0,7.0,1.0,0
4,4.0,1.0,1.0,3.0,2.0,1.0,3.0,1.0,1.0,0


In [65]:
from sklearn.datasets import load_digits
digits=load_digits()

In [66]:
dir(digits)

['DESCR', 'data', 'feature_names', 'frame', 'images', 'target', 'target_names']

In [67]:
df=pd.DataFrame(digits.data)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [68]:
df['target']=digits.target
df.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


In [69]:
x=df.drop(['target'],axis='columns')
y=digits.target

In [70]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

#### Teste com o Random Forest do sklearn

In [71]:
model = RandomForestClassifier(criterion='entropy')
model.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy')

In [72]:
model.score(X_test, y_test)

0.975

#### Teste 1 - Modelo basico

In [75]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

clf = RandomForest(n_trees=20)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)


KeyError: '[703, 1014, 957, 514, 195, 255, 511, 243, 92, 785, 399, 593, 211, 468, 1376, 1061, 1057, 1389, 165, 134, 643, 1186, 728, 1167, 1088, 887, 747, 1171, 1387, 1253, 687, 176, 650, 852, 632, 1254, 327, 343, 246, 831, 496, 1192, 251, 119, 121, 479, 334, 1326, 447, 172, 997, 581, 283, 1068, 492, 693, 777, 1181, 167, 984, 578, 1400, 612, 1359, 68, 1333, 163, 473, 1200, 73, 611, 938, 816, 1305, 1399, 1411, 99, 903, 1191, 559, 168, 345, 180, 130, 419, 694, 1170, 1355, 798, 991, 805, 488, 956, 1019, 425, 1051, 1256, 545, 1379, 566, 1244, 1418, 596, 173, 369, 982, 519, 360, 666, 344, 1073, 111, 886, 1304, 1231, 1122, 423, 569, 774, 781, 936, 527, 1290, 952, 536, 734, 232, 200, 671, 1109, 377, 450, 724, 754, 277, 125, 357, 1218, 1378, 787, 1383, 811, 575, 544, 495, 749, 1132, 266, 240, 436, 346, 307, 171, 597, 861, 409, 1374, 871, 1324, 752, 1024, 822, 1382, 296, 1183, 699, 1042, 837, 908, 215, 1161, 1077, 429, 86, 338, 904, 191, 1353, 306, 140, 1013, 336, 432, 995, 1406, 732, 379, 465, 

### DATASET 2 -

In [ ]:
#dataset = pd.DataFrame(arff.loadarff('datasets/xxxxxx.arff')[0])

### DATASET 3 -

In [ ]:
#dataset = pd.DataFrame(arff.loadarff('datasets/xxxxxx.arff')[0])